### Initial Steps

In [ ]:
import pandas as pd
import folium
path = 'C:\\Users\\ASUS\\projects\\Telkomsel_use_case\\datasets\\'

In [ ]:
df_train = pd.read_csv(path + "train.csv").drop(columns=['trip_duration', 'dropoff_datetime'])
df_test = pd.read_csv(path + "test.csv")
df = pd.concat([df_train, df_test], sort=False, ignore_index=True)

In [ ]:
from folium.plugins import HeatMapWithTime

Previewing the data that we will be working with

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

### Process Data

Generating month column

In [ ]:
df.pickup_datetime = pd.to_datetime(df.pickup_datetime, format='%Y-%m-%d %H:%M:%S')
df['month'] = df.pickup_datetime.apply(lambda x: x.month)
df['week'] = df.pickup_datetime.apply(lambda x: x.week)
df['day'] = df.pickup_datetime.apply(lambda x: x.day)
df['hour'] = df.pickup_datetime.apply(lambda x: x.hour)

In [ ]:
df.head()

In [ ]:
df.info()

As you can see we can obtain the month, week, day, and hour from the pickup_datetime column (which also gets converted to become a datetime column instead of object). We will use these columns in the later parts of our analysis.

### Map Visualizations with Folium

Generating the base map that will be used throughout this notebook


In [ ]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=11):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [ ]:
base_map = generateBaseMap()
base_map

In [ ]:
type(base_map)

In [ ]:
from folium.plugins import HeatMap

We will be using 2 months worth of data for this analysis, May - June 2016

In [ ]:
df_copy = df[df.month>4].copy()
df_copy['count'] = 1

In [ ]:
df_copy[['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().sort_values('count', ascending=False).head(10)

In [ ]:
base_map = generateBaseMap()
HeatMap(data=df_copy[['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

In [ ]:
base_map

In [ ]:
from folium.plugins import HeatMapWithTime

In [ ]:
df_hour_list = []
for hour in df_copy.hour.sort_values().unique():
    df_hour_list.append(df_copy.loc[df_copy.hour == hour, ['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().reset_index().values.tolist())

In [ ]:
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_hour_list, radius=5, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)
base_map